# Intro

- Combining Dataframes with pandas
- Data Workflows and Automation
- Plotting data

# Combining Dataframes with pandas
**45 min**
13:45 – 14:30

In many “real world” situations, the data that we want to use come in multiple files. We often need to combine these files into a single DataFrame to analyze the data. 

The pandas package provides various methods for combining DataFrames including `merge` and `concat`. More information in the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).

First need to load the two files into pandas DataFrames. We'll work with the surveys and species files

In [ ]:
import pandas as pd

In [ ]:
surveys_df = pd.read_csv("surveys.csv", 
                         keep_default_na=False, na_values=[""])

Take note that the `read_csv` method we used can take some additional options which we didn’t use previously. 

Many functions in Python have a set of options that can be set by the user if needed. In this case, we have told pandas to assign empty values in our CSV to NaN using `keep_default_na=False, na_values=[""]`.

In [ ]:
surveys_df.head()

We could specify a different value, for example `3`:

In [ ]:
surveys_nan = pd.read_csv("surveys.csv", 
                         keep_default_na=False, na_values=3)
surveys_nan.head()

This turns ever "3" into a NaN value.

Now we'll also bring in the species data that we talked about during the databases lesson:

In [ ]:
species_df = pd.read_csv("species.csv",
                         keep_default_na=False, na_values=[""])

In [ ]:
species_df.head()

## Concatenating DataFrames
We can use the concat function in pandas to append either columns or rows from one DataFrame to another. Let's split one of our dataframes into two to see how `concat` works.

In [ ]:
survey_sub_first = surveys_df.head(10)
survey_sub_last = surveys_df.tail(10)

If we look at the new `survey_sub_last` dataframe, we'll see that the index for each row hasn't changed.

In [ ]:
survey_sub_last

Notice how the "index" on this brand new dataframe doesn't start at '0' this is what I mean when I say the index label can be different from the actual positional index.


To change the index and have it start a 0 for this new dataframe instead of 35539, we'll need to use `reset_index`

In [ ]:
survey_sub_last = survey_sub_last.reset_index(drop=True)
survey_sub_last

**Challenge**: This is the second time we've used options, we used them earlier when importing the CSV file. Try making a new dataframe (`survey_sub_index`) with the last five rows of `survey_df` and resetting the index without the `drop=True` option to see what happens.

In [ ]:
# Challenge answer
survey_sub_index = surveys_df.tail(5)
survey_sub_index.reset_index()

**Challenge answer**: The `drop=True` option avoids adding new index column with old index values.

Now we have two new dataframes, `survey_sub_first` with the first 10 items from `surveys_df` and `survey_sub_last` with the last 10. And we've also reindexed `survey_sub_last`to start at 0. 

Next we'll do a vertical and horizontal concatenation of these two dataframes.

When we concatenate DataFrames, we need to use options again to specify the axis. 
- `axis=0` tells pandas to stack the second DataFrame under the first one. It will automatically detect whether the column names are the same and will stack accordingly.
- `axis=1` will stack the columns in the second DataFrame to the RIGHT of the first DataFrame.

Let's try this.

In [ ]:
vertical_stack = pd.concat([survey_sub_first, survey_sub_last], axis=0)

**Challenge**: Take a look at `vertical stack`. Notice anything unusual? 

In [ ]:
# Challenge answer
vertical_stack

**Challenge answer**: We see that the indexes repeat.

**Challenge**: Use `reset_index` to fix this.

In [ ]:
# Challenge answer
vertical_stack = vertical_stack.reset_index(drop=True)
vertical_stack

To concatenate these two dataframes horizontally, we use `axis=1` instead of `axis=0`

In [ ]:
horizontal_stack = pd.concat([survey_sub_first, survey_sub_last], axis=1)
horizontal_stack

## Joining Dataframes
When we concatenated our DataFrames we added them to each other - stacking them either vertically or side by side. 

Another way to combine DataFrames is to use columns in each dataset that contain common values (a common unique identifier). 

Combining DataFrames using a common field or column is called “joining”. The columns containing the common values are called “join key(s)”. **Note**: This process of joining tables is similar to what we do with tables in an SQL database.

Joining DataFrames in this way is often useful when one DataFrame is a “lookup table” containing additional data that we want to include in the other.

Let's go back to the two orginal dataframes we created `surveys_df` and `species_df`.

`species_df` was created from the species.csv file. This CSV is a lookup table. Let's take a glance at it.

In [ ]:
species_df.columns

This table contains the genus, species and taxa codes. How many species does it list?

In [ ]:
species_df.shape

It includes 55 species. These species are identified in our survey data as well using the unique species code.

Keeping the Surveys and Species data seperate makes sense for entering and storing data. Rather than adding 3 more columns for the genus, species and taxa to each of the 35,549 line Survey data table, we can maintain the shorter table with the species information. 

**Note**: Storing data in this way has many benefits including:

- It ensures consistency in the spelling of species attributes (genus, species and taxa) given each species is only entered once.
- It also makes it easy for us to make changes to the species information once without having to find each instance of it in the larger survey data.
- It optimizes the size of our data.

In [ ]:
surveys_df.head(5)

What if we want to add this species data to our existing `surveys_df` for analysis?

When we want to access that information, we can join the additional columns of information in Species to the Survey data. To better understand joins, let’s grab the first 10 lines of our Survey data as a subset to work with. We’ll use the `.head` method to do this.

In [ ]:
survey_sub = surveys_df.head(10)
species_sub = species_df.head(20)

`species_df` is the lookup table containing genus, species, and taxa names that we want to join with the data in `survey_sub` to produce a new DataFrame that contains all of the columns from both `species_df` and `survey_df`.

First, to identify appropriate join keys we first need to know which field(s) are the same between the DataFrames. We can inspect both DataFrames to identify these columns. 

If we are lucky, both DataFrames will have columns with the same name that also contain the same data. If we are less lucky, we need to identify a (differently-named) column in each DataFrame that contains the same information.

In [ ]:
survey_sub.columns

In [ ]:
species_sub.columns

In our example, the join key is the column containing the two-letter species identifier, which is called `species_id`.

Now that we know the fields with the common species ID attributes in each DataFrame, we are almost ready to join our data. 

But wait! There are different types of joins. We also need to decide which type of join makes sense for our analysis.

### Inner Join

The most common type of join is called an inner join. An inner join combines two DataFrames based on a join key and returns a new DataFrame that contains **only** those rows that have matching values in **both** of the original DataFrames.

*(Draw inner join)*

Inner joins yield a DataFrame that contains only rows where the value being joined exists in both tables.

In [ ]:
merged_inner = pd.merge(left=survey_sub, right=species_sub, left_on='species_id', 
                        right_on='species_id')

merged_inner.shape

*In this case `species_id` is the only column name in  both dataframes, so if we skipped `left_on` and `right_on` arguments we would still get the same result*

Notice that merged_inner has fewer rows than survey_sub. This is an indication that there were rows in `surveys_sub` with value(s) for `species_id` that do not exist as value(s) for species_id in `species_sub`.

Let's take a look at our new dataframe.

In [ ]:
merged_inner

## Left Join

What if we want to add information from `species_sub` to `survey_sub` without losing any of the information from `survey_sub`? In this case, we use a different type of join called a “left outer join”, or a “left join”.

Like an inner join, a left join uses join keys to combine two DataFrames. Unlike an inner join, a left join will return **all** of the rows from the left DataFrame.

This includes rows whose join key(s) do not have values in the right DataFrame. Rows in the left DataFrame that are missing values for the join key(s) in the right DataFrame will simply have null (i.e., NaN or None) values for those columns.

**Note**: a left join will still discard rows from the right DataFrame that do not have values for the join key(s) in the left DataFrame.

In [ ]:
# This time we specify 'how'
merged_left = pd.merge(left=survey_sub,right=species_sub, how='left', 
                       left_on='species_id', right_on='species_id')

merged_left.shape

This time we have the same number of rows as `survey_sub`. And we can take a look at the result.

In [ ]:
merged_left

The pandas `merge` function supports two other join types:

- Right (outer) join: Invoked by passing how='right' as an argument. Similar to a left join, except all rows from the right DataFrame are kept, while rows from the left DataFrame without matching join key(s) values are discarded.

- Full (outer) join: Invoked by passing how='outer' as an argument. This join type returns the all pairwise combinations of rows from both DataFrames; i.e., the result DataFrame will NaN where data is missing in one of the dataframes. This join type is very rarely used.

(NEEDS WORK)
**Challenge**: Create a new DataFrame by joining the contents of the full surveys_df and species_df. Which join makes the most sense for analyzing the Survey data?

Then count the number of taxa per plot?

In [ ]:
merged = pd.merge(left=surveys_df,right=species_df, how='left', left_on='species_id', right_on='species_id')

In [ ]:
grouped_data = merged.groupby('plot_id')['taxa'].count()

In [ ]:
grouped_data

## Key Points

- Pandas’ `merge` and `concat` can be used to combine subsets of a DataFrame, or even data from different files.
- Joining two DataFrames with `merge` can be done in multiple ways (left, right, and inner) depending on what data must be in the final DataFrame.